<a href="https://colab.research.google.com/github/purvasingh96/Deep-learning-with-neural-networks/blob/master/Deep-learning-with-pytorch/3.%20Recurrent%20Neural%20Networks/Char_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN


This notebook demonstrates how to generate text using a **charecter-level LSTM with PyTorch** using dataset from the book **Anna Karenina**. Given a sequence of charecter from this book, the model will generate longer sequences of data by calling the model repeatedly.

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure. Below is the **general architecture of the character-wise RNN.**<br>
<img src="https://github.com/purvasingh96/Deep-learning-with-neural-networks/blob/master/Deep-learning-with-pytorch/3.%20Recurrent%20Neural%20Networks/images/lstm_rnn_architecture.png?raw=1"></img> 




# Set Up
### Import PyTorch and other libraries


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np


# Download the Anna Karenina data
 

### Read the data

In [0]:
with open('sample_data/anna.txt', 'r') as f:
  text = f.read()

### First look at the text

In [16]:
print(text[:100])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin
